# RoBERTa full fine-tune (Kaggle)

This notebook runs a full RoBERTa-base fine-tune for the TEXT_BRANCH on Kaggle.

Usage notes:
- Upload `data/iemocap_manifest.jsonl` as a Kaggle Dataset and add it to the Notebook (recommended).
- Use GPU Accelerator (T4 or better).
- Adjust `--batch_size`, `--accumulation_steps`, and `--max_length` depending on GPU memory.

In [ ]:
!pip install -q -U --upgrade-strategy only-if-needed "transformers==4.57.1" "accelerate>=1.9.0" "datasets>=2.14,<3.0" "evaluate>=0.4" "sentencepiece>=0.1.99" "sentence-transformers>=3.0,<4.0"

In [ ]:
import torch, transformers, accelerate, datasets, numpy as np
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)
print("datasets:", datasets.__version__)
print("numpy:", np.__version__)

In [ ]:
!git clone https://github.com/SpeedyLabX/ser-conformer-gat-xai.git
%cd ser-conformer-gat-xai

In [ ]:
pip install -e . --no-deps

In [ ]:
!mkdir -p data
!cp /kaggle/input/slx02-ser-dataset/iemocap_manifest.jsonl data/iemocap_manifest.jsonl
!ls -l data || true

In [ ]:
!WANDB_DISABLED=true WANDB_SILENT=true HF_HUB_DISABLE_TELEMETRY=1 \
python scripts/finetune_roberta.py \
  --manifest data/iemocap_manifest.jsonl \
  --backbone roberta-base \
  --batch_size 8 --accumulation_steps 4 --epochs 100 \
  --max_length 128 --num_class 6 \
  --out_dir /kaggle/working/artifacts/roberta_kaggle \
  --fp16 --gradient_checkpointing \
  --load_best_model_at_end \
  --evaluation_strategy epoch \
  --save_total_limit 5 \
  --early_stopping_patience 5 \
  --lr 1e-5 --use_tqdm \
  --split_by_session --train_sessions 1,2,3,4 --test_sessions 5

In [ ]:
python scripts/evaluate_text.py \
  --manifest data/iemocap_manifest.jsonl \
  --checkpoint /kaggle/working/artifacts/roberta_kaggle \
  --out_dir /kaggle/working/artifacts/roberta_kaggle \
  --save_confusion_plot

In [ ]:
python scripts/evaluate_text.py \
  --manifest data/iemocap_manifest.jsonl \
  --checkpoint /kaggle/working/artifacts/roberta_kaggle \
  --out_dir /kaggle/working/artifacts/roberta_kaggle \
  --save_confusion_plot --map_to_4class